### MIP Problem Definition

* Sets: J (Orders/Missions), I (Operators), T (Order types)
* Variables: $x_{ij}$ (Assignment), $S_{j}$ (Start Time), ${Cj}$ (Completion Time), Z (Makespan)	  
Sequencing variable: $z_{ikj}$ (operator i goes from order k to order j)
* Parameters:
  - $P_{ij}$(Processing time) => time of loading and unloading of pallet j.
  - $T{jk}$ (Travel time from j to k) => calculated basing on any possible.combination between two orders i and k.
  - $S{it}$ (Skill score) => the score that a fork_lift is adequate to pallet dimensions "less loss in fork space".
  - $O{jt}$ (Order type) => could be ignored in the future.
  - M (Big number)

#### Constaints and Objective function

1. Each order j is assigned to a single operator i
$$\sum_{i \in I} x_{i j} = 1 \quad \forall j \in J$$

2. The type of order j should be handled by an operator i that has skill score >= 1 for such order
 $$\sum_{t \in T} O_{j t} \cdot S_{i, t} \ge x_{i j} \quad \forall i \in I, j \in J$$

3. The linking between order k and order j over the same operator i
 $$\sum_{k \in J \cup \{\text{Base}\}} z_{i k j} = x_{i j} \quad \forall i \in I, j \in J$$
(The operator i must travel to j if and only if i is assigned to j.)

4. the number of flow-ins (j->k) should be equal to the number flow-outs (l->j) of the same operator i for any order j
$$\sum_{k \in J \cup \{\text{Base}\}} z_{i j k} = \sum_{l \in J \cup \{\text{Base}\}} z_{i l j} \quad \forall i \in I, j \in J$$
(Including both directions towards the base)

5. Time sequencing (the start time of order k is after the completion time of the previous linked order j + the travel time to reach order k)
$$S_k \ge C_j + T_{j k} - M \cdot (1 - z_{i j k}) \quad \forall i \in I, j, k \in J, j \ne k$$
(M is a big number that activate the constraint in case of linking order k and order j; otherwise, the constraint give a very large negative number)

6. The completion time of order j is the time to start it + the processing time w.r.t. the assigned operator i
$$C_j = S_j + \sum_{i \in I} x_{i, j} \cdot P_{i j} \quad \forall j \in J, i \in I$$
7. Common MTZ (Miller-Tucker-Zemlin) constraints to prevent isolated loops/subtours
 $$u_j - u_k + |J| \cdot z_{i j k} \le |J| - 1 \quad \forall i \in I; j, k \in J, j \ne k$$
(Where $u_{j}$ and $u_{k}$ are helper variables representing the sequence position of j and k on the route,   
and $2 \le u_{j} \le |J|+1$.)

8. Objective function (minimize the makespan for any order j)
 $$Z \ge C_j \quad \forall j \in J$$

*Base order/mission is at position [0, 0], from which all operator should start the day.

***Fact***: we need to calculate the travel time "$T_{jk}$" between any possibile combination of two consecutive orders/missions.  
***Deduction***: the search space "possibile routes of order pairs" for the MIP solver will grow exponentially as the number of input orders grows,
which could lead to a combinatorial explosion of the possibile route sequences to be optimized.
Therefore, the problem can't be solved with direct MIP in affordable time in case of very large number of orders/missions (NP-hard problem affected by the curse of dimensionality).  
Note that in our case the batch has 10,000 orders in average.


So we need to decompose the problem into subproblems in order to solve it.

### Direct MIP Alternative Solutions

* Possibile approaches:
1. Clustering + MIP: (advantage in concurrent subproblem solving)

    - cluster the thousends of missions into K clusters (ex. based on spatial features)
    - run the MIP for each cluster (ex. should be in hundreds) to solve the cluster sequences per operators
    - aggregate the sequences the whole sequences over operators.  

    Drawback: the global suboptimality might require the overlapping between clusters!

2. Simplified MIP + Metaheuristics: (advantage in concurrent subproblem solving)   
    - set up initial MIP that only decides which operator gets which batch of orders w.r.t. the whole amount orders,
    ignoring the detailed sequencing and subtour constraints,
    aiming to minimize the total workload imbalance and makespan.
    - apply metaheuristic algorithm (ex. genetic) for each operator batch in order to obtain a suboptimal operator overall time minimizing the makespan.  

    Drawback: the results of the metaheuristic algorithm are based on the initial distribution of orders per each operator, which is determined by
    the simplified MIP not aware of full sequencing constraints.

3. MIP + GNN + Greedy (to be explored further):  
    - execute a MIP solver on a mini batch with a small number of orders and operators,
    then the output (operator-order assignments) will be used as target labels for the specified input.
    - a GNN is trained on the labeled examples produced at the first point, so it produces operator-order priorities/sequences scores.
    - greedy algorithm to collect the hightest-scoring unassigned orders per operator.  

    Drawback: the GNN might generalize the patterns learned only from the small number of labeled examples in the training phase;
    therefore, it could be biased on the trained examples.
    Remains also necessary to show the explainability of GNN outputs.
    

### The scheduling (ex. the sequence of orders/missions per operator) is important? should it be performed during orders assignment per operator?

The efficiency of an assignment is determined by its optimal position in a certain sequence of missions per operator.  
Example: if we arbitrarily assign an order x at [50, 60] nord east of the warehouse with travel time $T_{base, x}$ = 4 min, and the second order in the sequence is order y at [-10, 20] nord west of the warehouse with travel time $T_{base, y}$ = 2 min, the operator has consumed inefficiently other 4 min of travel to return to the base in order to pick up order y, supposing that $T_{base, x}$ = $T_{x, base}$ = 4 min, so the overall makespan in this case => 4 + 4 + 2 = 10 min.
Meanwhile if we assign order y then order x consecutively, the overall makespan in this case => 2 + 2 + 4 = 8 min, supposing that $T_{base, y}$ = $T_{y, base}$ = 4 min, so we've efficiently minimized the overall makespan per such operator.

Since the quality of the assignment is determined by the sequence to which it belongs, we should perform the assignment based on the optimal scheduling/sequencing.

### Additions
+ the set of operators is unknown "to be defined by the solver".
+ as parameter, there'll be the number of work hours per operator.
+ we might need to optimize the number of operators to be used.
+ define an index to measure the difference between the optimal and the sub-optimal solutions.
+ [optional] optimize the combination of missions per the same operator "the operator can load/unload various pallets togther".
+ [optional] use GIS to represent the spatial warehouse map and accurately estimate the shortest distance between any two points.

### Trade-offs
- In case of separated MIP solvers [one for operator activation and one for scheduling & mission assignment]: Isolating the operator activation MIP problem from the scheduling MIP problem, might generate overscheduled orders per the activated operators? in the other hand, if we merge both problems in one MIP, the objective function wouldn't be focalized only on the scheduling or on the operator activation.  

  Why Overscheduling might occur:  
  The preliminary activation MIP solves a static, simplified problem:  
  $$\text{Minimize } \sum_{i \in I} y_{i} \text{ which is subject to } \sum_{j} x_{ij}⋅P_{ij} \le H_{fixed}$$ where $y_{i}$ is the activation variable of operator i and $H_{fixed}$ is the number of fixed work hours (ex. 20 hours).

    * Blindness to Travel Time: The first MIP solver looks only at processing time ($P_{ij}$). It determines the minimum number of operators necessary to cover the workload hours. It assumes travel time ($T_{jk}$) is either zero or a negligible constant.

    * Overscheduled Output: When we pass the result (the minimum number of operators) to the second Scheduling MIP/GNN, the travel time suddenly becomes a factor. That "minimal" team might find that the required routing and travel time pushes their total shift length well past the $H_{fixed}$ limit, or critically, pushes the Makespan (Z) to an unacceptable level.

- In case of combined MIP solvers [only one solver for operatora activation, scheduling and mission assignment]: the objective function would be a weighted combination for a multi-objective problem.
$$\text{Minimize } Z_{\text{Total}} = \left( \alpha \cdot Z_{\text{Makespan}} \right) + \left( \beta \cdot \sum_{i \in I} y_i \right)$$

    * Enforcing the Trade-off: The single objective function forces the solver (MIP or GNN) to find a solution where the marginal benefit of reducing time ($Z$) is perfectly balanced against the marginal cost of labor ($\sum y_i$), according to your chosen weights ($\alpha$ and $\beta$).

    * The Solver's Focus: The solver is still entirely "focalized" but its focus is on achieving the best possible compromise.If $\alpha$ is large and $\beta$ is small, the focus is almost entirely on scheduling ($Z$).If $\beta$ is large and $\alpha$ is small, the focus is almost entirely on activation ($\sum y_i$).  

  => But determining the optimal alpha and beta is a second problem of optimality!  
   Determining the optimal weights ($\alpha$ and $\beta$) is often considered a separate, higher-level optimization problem, it moves from tactical scheduling to strategic decision-making.
    * $\beta$ (Labor Cost): This is usually a fixed, hard cost. It represents the financial penalty associated with activating one additional full-time operator (e.g., salary, benefits, overhead, per hour or per shift).
    * $\alpha$ (Time Cost): This is the cost associated with one unit of Makespan time (e.g., the cost of delaying downstream processes, lost throughput opportunity, inventory holding cost, or customer service penalty).  

  Example: If hiring one operator costs 50 euro per hour, and the penalty for delaying the makespan by one hour is judged to be 100 euro, then:
  $$\beta = 50, \alpha = 100$$
  Obviously, it might be needed to normalize them.  
  Some techniques are used to explore the solution space and guide the decision-maker:
    * The Epsilon-Constraint Method: it generates the "Pareto Front", a curve showing the minimum number of operators required for any given makespan target. The decision-maker then visually chooses the point on the curve that best fits the company's budget and service goals.
    * Utility Function Elicitation: involves working with stakeholders to define a business strategy or Analytic Hierarchy Process (AHP) to formally decide the relative importance of labor cost vs. time cost.